## **Installing Ultralytics for Using Pre-trained Yolo Models**

In [1]:
# Install the ultralytics package from PyPI
!pip install ultralytics
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.8/883.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 1.9 MB/s eta 0:00:00


# **Importing Used Libraries and Seed for Reproducibility**

In [2]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import random
import torch ## PyTorch
from itertools import islice
from ultralytics import YOLO
import pandas as pd
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import shutil
from PIL import Image


## For selective classification
import warnings
# Suppress the specific sklearn warning
warnings.filterwarnings("ignore", message= "y_pred contains classes not in y_true")
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Setting the random seed for reproducibility
seed = 0
np.random.seed(seed) ## for numpy
torch.manual_seed(seed) ## for PyTorch
torch.cuda.manual_seed(seed)
random.seed(0) ## for random module on python


# Check if a GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("GPU is not available.")
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") ## use gpu if its available

GPU is not available.


# **1 - Loading and Preparing the Data (EDA)**

In [4]:
## uses the kitti-dataset-yolo-format file to convert to acceptable yolo inputs
base_dir = Path('/kaggle/input/kitti-dataset')
img_path = base_dir / 'data_object_image_2' / 'training' / 'image_2'
label_path = Path('/kaggle/input/kitti-dataset-yolo-format/labels')
with open('/kaggle/input/kitti-dataset-yolo-format/classes.json','r') as f:
    classes = json.load(f)

classes

{'Car': 0,
 'Pedestrian': 1,
 'Van': 2,
 'Cyclist': 3,
 'Truck': 4,
 'Misc': 5,
 'Tram': 6,
 'Person_sitting': 7}

In [5]:
## create pais with images and labels files
images = sorted(list(img_path.glob('*')))
labels = sorted(list(label_path.glob('*')))
pairs = list(zip(images,labels))
pairs[:2]

[(PosixPath('/kaggle/input/kitti-dataset/data_object_image_2/training/image_2/000000.png'),
  PosixPath('/kaggle/input/kitti-dataset-yolo-format/labels/000000.txt')),
 (PosixPath('/kaggle/input/kitti-dataset/data_object_image_2/training/image_2/000001.png'),
  PosixPath('/kaggle/input/kitti-dataset-yolo-format/labels/000001.txt'))]

In [6]:
## spliting between train/val/test sets

train, test_val = train_test_split(pairs,test_size=0.2,shuffle=True)

val, test = train_test_split(test_val, test_size=0.1, shuffle = False)

len(train), len(val), len(test)

(5984, 1347, 150)

In [7]:
## creating paths for train/val/test sets (organizing files)

# destination paths
output_dir = Path('/kaggle/working')
output_train = output_dir / 'train'
output_val = output_dir / 'val'
output_test = output_dir / 'test'

In [8]:
## creating the directories (train/val/test)
output_train.mkdir(parents=True, exist_ok=True)
output_val.mkdir(parents=True, exist_ok=True)
output_test.mkdir(parents=True, exist_ok=True)

In [9]:
## creating function to copy files
def copy_files(pairs, output_folder):
    for img_file, label_file in pairs:
        shutil.copy(img_file, output_folder / img_file.name)
        shutil.copy(label_file, output_folder / label_file.name)

In [10]:
## copying to their respective directories
copy_files(train, output_train)
copy_files(val, output_val)
copy_files(test, output_test)

print(f'Training dataset: {len(train)} samples')
print(f'Validation dataset: {len(val)} samples')
print(f'Test dataset: {len(test)} samples')

Training dataset: 5984 samples
Validation dataset: 1347 samples
Test dataset: 150 samples


In [11]:
## creating YAML file for to point to the data
yaml_file = 'names:\n'
yaml_file += '\n'.join(f'- {c}' for c in classes)
yaml_file += f'\nnc: {len(classes)}'
yaml_file += f'\ntrain: {str(output_train)}\nval: {str(output_val)} \ntest: {str(output_test)}'
with open('kitti.yaml','w') as f:
    f.write(yaml_file)

In [12]:
!cat kitti.yaml

names:
- Car
- Pedestrian
- Van
- Cyclist
- Truck
- Misc
- Tram
- Person_sitting
nc: 8
train: /kaggle/working/train
val: /kaggle/working/val 
test: /kaggle/working/test

# **2 - Choosing Model, Training/Validation Steps**

## **Instantiating the Model**

In [13]:
# Load a model
model = YOLO("yolo11n.yaml")  # build a new model from YAML
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

100%|██████████| 5.35M/5.35M [00:00<00:00, 63.3MB/s]


Transferred 499/499 items from pretrained weights


## **Training**

In [14]:
#train_results = model.train(
#    data='/kaggle/working/kitti.yaml', 
#    epochs=10,
#    patience=3,
#    mixup=0.1,
#    project='yolo11n-kitti',
#    device=0
#)

## **Validating**

In [15]:
#val_results = model.val()

## **Seeing Results from Training**

In [16]:
#plt.figure(figsize=(10,20))
#plt.imshow(Image.open('/kaggle/working/yolo11n-kitti/train/results.png'))
#plt.axis('off')
#plt.show()

In [17]:
#plt.figure(figsize=(10,20))
#plt.imshow(Image.open('/kaggle/working/yolo11n-kitti/val/confusion_matrix.png'))
#plt.axis('off')
#plt.show()